In [6]:
dic = {'a': 3, 'b': 2}

In [8]:
li = ['a', 'a', 'a', 'b', 'b']

In [9]:
# convert element of li by dic
li = [dic[i] for i in li]

In [10]:
import numpy as np

In [12]:
hand = np.random.rand(12, 543, 2)

In [13]:
hand[5, 52, 1] = np.nan

array([0.50411222, 0.50209743, 0.50199629, 0.48889409, 0.48850127,
              nan, 0.51635373, 0.49183435, 0.50153748, 0.48732868,
       0.50116818, 0.49980379])

In [17]:
# delete nan frame
not_nan_frame = ~np.isnan(np.mean(hand, axis=(1, 2)))
hand = hand[not_nan_frame]

In [19]:
hand.shape

(11, 543, 2)

In [1]:
from torchinfo import summary

In [41]:
from model_ref import ModelFixedLen
def create_model():
    model = ModelFixedLen(
        in_dim_H=42,
        in_dim_L=80,
        dim1_H=128,
        dim1_L=64,
        dim2=512,
        n_class=59,
    )
    return model

In [43]:
model = create_model()

summary(
    model,
    input_size=((1, 42, 576), (1, 80, 784)),
)

Layer (type:depth-idx)                   Output Shape              Param #
ModelFixedLen                            [1, 59]                   --
├─BackboneFixedLen: 1-1                  [1, 512]                  --
│    └─BatchNorm1d: 2-1                  [1, 42, 576]              84
│    └─Conv1d: 2-2                       [1, 128, 576]             16,256
│    └─RSUnit1D: 2-3                     [1, 128, 576]             49,408
│    │    └─Conv1d: 3-1                  [1, 128, 576]             49,152
│    │    └─BatchNorm1d: 3-2             [1, 128, 576]             256
│    └─RSUnit1D: 2-16                    --                        (recursive)
│    │    └─LeakyReLU: 3-3               [1, 128, 576]             --
│    └─RSUnit1D: 2-5                     --                        (recursive)
│    │    └─Conv1d: 3-4                  [1, 128, 576]             49,152
│    │    └─BatchNorm1d: 3-5             [1, 128, 576]             256
│    └─MaxPool1d: 2-6                    [1, 128,

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Model(nn.Module):
    def __init__(self, seq_len, n_features, n_class):
        super(Model, self).__init__()
        self.bn0 = nn.BatchNorm1d(n_features)
        self.conv1 = nn.Conv1d(n_features, 128, 3, padding=1)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc = nn.Linear(128, n_class)
    
    def forward(self, x):
        '''
            input: (bs, 42, 576)
            output: (bs, 576(seq_len), 59(n_classes))
        '''
        # x = self.bn0(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x))) # (bs, 42, 576)
        x = x.permute(0, 2, 1) # (bs, 576, 42)
        x = self.fc(x) # (bs, 42, 59)
        return x

In [4]:
model = Model(574, 42, 59)

In [5]:
summary(model, input_size=(1, 42, 574))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 574, 59]              84
├─Conv1d: 1-1                            [1, 128, 574]             16,256
├─BatchNorm1d: 1-2                       [1, 128, 574]             256
├─Conv1d: 1-3                            [1, 128, 574]             49,280
├─BatchNorm1d: 1-4                       [1, 128, 574]             256
├─Linear: 1-5                            [1, 574, 59]              7,611
Total params: 73,743
Trainable params: 73,743
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 37.63
Input size (MB): 0.10
Forward/backward pass size (MB): 2.62
Params size (MB): 0.29
Estimated Total Size (MB): 3.01

In [71]:
bn = nn.BatchNorm1d(42)
x = torch.randn(1, 42, 574)

In [72]:
out = bn(x)

In [76]:
out.shape

torch.Size([1, 42, 574])

In [77]:
out[0, 1, :].std()

tensor(1.0009, grad_fn=<StdBackward0>)

In [32]:
out[0, 2, :].mean()

tensor(-0.0134, grad_fn=<MeanBackward0>)

In [33]:
out.shape

torch.Size([1, 12, 1086])

In [9]:
from Levenshtein import distance 
import numpy as np

In [10]:
distance('aab', 'aa')

1

In [13]:
distance(np.array([[2,2,3]]), np.array([[3, 4]]))

TypeError: unhashable type: 'numpy.ndarray'